In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

Input:
```sql
time,customer_key,branch,user,pc_name,pc_user,command_type,command_name
2021-12-01T00:00:05.774,2108386,branch1663,user19493,pcName26646,pcUser29,プロセス,login
2021-12-01T00:00:05.775,2108386,branch1663,user19493,pcName26646,pcUser29,プロセス,start
2021-12-01T00:00:09.653,2101030,branch2156,user21542,pcName21710,pcUser21544,帳票出力,物件チラシ_賃貸借
2021-12-01T00:00:16.331,2002204,branch1368,user8737,pcName17785,pcUser17786,データ操作,物件 > 賃貸募集一覧 > 部屋コピー
2021-12-01T00:00:25.220,2002204,branch1055,user5557,pcName5558,pcUser5559,検索,賃貸募集一覧 > 再読込
2021-12-01T00:00:34.821,2101030,branch2156,user21542,pcName21710,pcUser21544,帳票出力,物件チラシ_賃貸借
2021-12-01T00:00:40.830,2100291,branch1537,user22168,pcName23063,pcUser23064,プロセス,end
```

Output:
```sql
session_id;user_id;item_id;time_stamp
1;NA;35077;390072;2016-05-09T16:47:49
1;NA;36118;487369;2016-05-09T16:47:49
1;NA;129055;991416;2016-05-09T16:47:49
2;NA;3147;883039;2016-05-09T16:47:49
2;NA;32754;359839;2016-05-09T16:47:49
2;NA;100747;38317;2016-05-09T16:47:49
2;NA;32971;182759;2016-05-09T16:47:49
```

In [3]:
from datetime import datetime
time = datetime.now()
time.strftime('%Y-%m-%dT%H:%M:%S')

'2023-06-09T16:47:49'

In [2]:
path = 'downloads/202112userlogs.csv'
userlogdf = pd.read_csv(path)
# time,customer_key,branch,user,pc_name,pc_user,command_type,command_name

In [4]:
time = pd.to_datetime(userlogdf['time'])
userlogdf['time'] = time

## 一意のユーザID再振り分け

In [73]:
userlogdf.shape

(7956006, 8)

In [5]:
user_df = userlogdf[['customer_key', 'user', 'pc_name', 'pc_user']]
user_df = user_df[~user_df.duplicated()]
# print(user_df.shape)
# print(user_df['pc_user'].unique().shape)
# print(user_df['pc_name'].unique().shape)
# print(user_df['user'].unique().shape)
user_id = list(range(1, len(user_df)+1))
# print(len(user_id))
user_id_map = dict()
for i, v in enumerate(user_df[['customer_key', 'user', 'pc_name', 'pc_user']].values):
    # print(tuple(v))                  
    user_id_map[tuple(v)] = i + 1
user_id = []
for row in userlogdf[['customer_key', 'user', 'pc_name', 'pc_user']].itertuples():
    map_key = tuple(row)[1:] # remove [0]index
    user_id.append(user_id_map[map_key])
print(len(user_id))

7956006


In [22]:
user_df[0:1].dtypes

customer_key     int64
user            object
pc_name         object
pc_user         object
user_id          int64
dtype: object

In [40]:
# test
map = {
    (2108386, 'user19493', 'pcName266646', 'pcUser29'): 1
}
map[(2108386, 'user19493', 'pcName266646', 'pcUser29')]
user_df[0:1][['customer_key', 'user', 'pc_name', 'pc_user']].iloc[0]
map[tuple(user_df[0:1][['customer_key', 'user', 'pc_name', 'pc_user']].iloc[0])]

customer_key        2108386
user              user19493
pc_name         pcName26646
pc_user            pcUser29
Name: 0, dtype: object

## 一意のコマンドIDの割り振り

In [6]:
command_df = userlogdf[['command_type', 'command_name']]
command_df = command_df[~command_df.duplicated()]
command_id = list(range(1, len(command_df)+1))
# print(len(user_id))
command_id_map = dict()
for i, v in enumerate(command_df[['command_type', 'command_name']].values):
    # print(tuple(v))
    command_id_map[tuple(v)] = i + 1
command_id = []
for row in userlogdf[['command_type', 'command_name']].itertuples():
    map_key = tuple(row)[1:] # remove [0]index
    command_id.append(command_id_map[map_key])
print(len(command_id))

7956006


## セッション番号の割り振り

In [13]:
prepreprocess_df = pd.DataFrame(np.array([user_id, command_id]).T, columns=['user_id', 'command_id'])
prepreprocess_df['timestamp'] = userlogdf.time
prepreprocess_df.sort_values('user_id', inplace=True)

In [14]:
prepreprocess_df

,user_id,command_id,timestamp
0,1,1,2021-12-01 00:00:05.774
7699987,1,2,2021-12-27 21:12:37.645
7699989,1,1,2021-12-27 21:12:38.419
7699990,1,301,2021-12-27 21:12:38.424
2304813,1,16,2021-12-11 21:36:49.424
...,...,...,...
7955634,30175,298,2021-12-31 20:35:10.548
7955650,30176,7,2021-12-31 20:38:15.307
7955842,30177,306,2021-12-31 21:29:22.718
7955880,30178,298,2021-12-31 21:45:19.195


In [22]:
# test
(prepreprocess_df.iloc[1]['user_id'] != prepreprocess_df.iloc[0]['user_id'])

False

In [27]:
session_id = [1]
cur_session_id = 1
pre_row = prepreprocess_df.iloc[0]
PIVOT_SECOND = 15 * 60
for row in prepreprocess_df[1:].itertuples():
    # 0:index, 1: user_id, 2:command_id, 3:timestamp
    # print(row)
    # print(row.timestamp)
    if (row.timestamp - pre_row.timestamp).total_seconds() > PIVOT_SECOND or row.user_id != pre_row.user_id:
        cur_session_id += 1
    session_id.append(cur_session_id)
    pre_row = row
# prepreprocess_df['session_id'] = session_id

In [28]:
print(cur_session_id)

3790028


In [31]:
prepreprocess_df.insert(0, 'session_id', session_id)

In [32]:
prepreprocess_df

,session_id,user_id,command_id,timestamp
0,1,1,1,2021-12-01 00:00:05.774
7699987,2,1,2,2021-12-27 21:12:37.645
7699989,2,1,1,2021-12-27 21:12:38.419
7699990,2,1,301,2021-12-27 21:12:38.424
2304813,2,1,16,2021-12-11 21:36:49.424
...,...,...,...,...
7955634,3790024,30175,298,2021-12-31 20:35:10.548
7955650,3790025,30176,7,2021-12-31 20:38:15.307
7955842,3790026,30177,306,2021-12-31 21:29:22.718
7955880,3790027,30178,298,2021-12-31 21:45:19.195


In [51]:
# test
# for low in userlogdf[]
# userlogdf[1].time - userlogdf[0].time
delta = userlogdf.iloc[1].time - userlogdf.iloc[0].time
delta.total_seconds()

0.001

## 組み合わせます

In [78]:
# session_id;user_id;item_id;timestamp
prepreprocess_df.to_csv('./SR-GNN/datasets/eseikatsu.csv')

NameError: name 'session_id' is not defined